# Comparison: Graphix to Guppy vs Graphix to pytket

This notebook compares two approaches to converting Graphix MBQC patterns:

1. **Original (graphix_to_guppy.py)**: Generates Python code with explicit `if` statements for conditional corrections
2. **pytket approach (graphix_to_pytket.py)**: Uses pytket's native `condition` argument for conditional gates

## Key Differences

### Original Approach
- Generates imperative code with control flow (`if` statements)
- XOR conditions computed at runtime as boolean expressions
- Output: String of Python/Guppy code

### pytket Approach
- Uses declarative conditional gate mechanism
- XOR conditions expressed as pytket `BitLogicExp` objects
- Output: pytket `Circuit` object with metadata

Let's compare them on various test cases!

In [1]:
# Install required packages (uncomment if needed)
# !pip install graphix pytket numpy

In [2]:
import numpy as np
from graphix import Circuit as GraphixCircuit
from graphix.pattern import Pattern

# Import both converters
import sys
sys.path.append('.')

from graphix_to_guppy import convert_graphix_pattern_to_guppy
from graphix_to_pytket import convert_graphix_pattern_to_pytket

from pytket.circuit.display import render_circuit_jupyter

print("✓ All imports successful!")

✓ All imports successful!


## Test 1: Simple Single-Qubit Gate (Hadamard)

A simple circuit to see basic structure differences.

In [3]:
# Create a simple Hadamard circuit
circuit1 = GraphixCircuit(1)
circuit1.h(0)
pattern1 = circuit1.transpile().pattern

print("Graphix Pattern Commands:")
for i, cmd in enumerate(pattern1):
    print(f"  {i}: {cmd}")
print()

Graphix Pattern Commands:
  0: N(1)
  1: E((0, 1))
  2: M(0)
  3: X(1, {0})



### Original Approach: Generated Guppy Code

In [4]:
guppy_code1 = convert_graphix_pattern_to_guppy(pattern1)
print("="*70)
print("GUPPY CODE OUTPUT:")
print("="*70)
print(guppy_code1)
print("="*70)

GUPPY CODE OUTPUT:
from guppy import guppy
from guppy.prelude.quantum import qubit, measure, h, x, y, z, s, sdg, rx, ry, rz, cz

@guppy
def quantum_circuit(q_in_0: qubit) -> tuple[qubit, bool]:
    q_0 = qubit()  # Allocate qubit in |0⟩
    q_0 = h(q_0)  # Prepare |+⟩ state
    q_in_0, q_0 = cz(q_in_0, q_0)
    q_in_0 = h(q_in_0)
    m_1 = measure(q_in_0)
    if m_1:
        q_0 = x(q_0)
    return q_0, m_1


### pytket Approach: Circuit with Condition Arguments

In [5]:
pytket_circ1 = convert_graphix_pattern_to_pytket(pattern1)

print("="*70)
print("PYTKET CIRCUIT OUTPUT:")
print("="*70)
print(f"Qubits: {pytket_circ1.n_qubits}")
print(f"Bits: {pytket_circ1.n_bits}")
print(f"Gates: {pytket_circ1.n_gates}")
print(f"Depth: {pytket_circ1.depth()}")
print("\nCommands:")
for i, cmd in enumerate(pytket_circ1.get_commands()):
    print(f"  {i}: {cmd}")
print("="*70)

# Visualize the circuit
render_circuit_jupyter(pytket_circ1)

PYTKET CIRCUIT OUTPUT:
Qubits: 2
Bits: 1
Gates: 5
Depth: 5

Commands:
  0: H q[1];
  1: CZ q[0], q[1];
  2: H q[0];
  3: Measure q[0] --> m[0];
  4: IF ([m[0]] == 1) THEN X q[1];


## Test 2: Circuit with Clifford Gate (S gate)

Testing how Clifford gates are decomposed.

In [6]:
circuit2 = GraphixCircuit(1)
circuit2.s(0)
pattern2 = circuit2.transpile().pattern

print("Graphix Pattern Commands:")
for i, cmd in enumerate(pattern2):
    print(f"  {i}: {cmd}")
print()

Graphix Pattern Commands:
  0: N(1)
  1: N(2)
  2: E((0, 1))
  3: E((1, 2))
  4: M(0, angle=-0.5)
  5: M(1)
  6: X(2, {1})
  7: Z(2, {0})



In [7]:
# Original approach
guppy_code2 = convert_graphix_pattern_to_guppy(pattern2)
print("="*70)
print("GUPPY CODE:")
print("="*70)
print(guppy_code2)
print("="*70)

GUPPY CODE:
from guppy import guppy
from guppy.prelude.quantum import qubit, measure, h, x, y, z, s, sdg, rx, ry, rz, cz

@guppy
def quantum_circuit(q_in_0: qubit) -> tuple[qubit, bool, bool]:
    q_0 = qubit()  # Allocate qubit in |0⟩
    q_0 = h(q_0)  # Prepare |+⟩ state
    q_1 = qubit()  # Allocate qubit in |0⟩
    q_1 = h(q_1)  # Prepare |+⟩ state
    q_in_0, q_0 = cz(q_in_0, q_0)
    q_0, q_1 = cz(q_0, q_1)
    q_in_0 = rz(q_in_0, 0.5)
    q_in_0 = h(q_in_0)
    m_2 = measure(q_in_0)
    q_0 = h(q_0)
    m_3 = measure(q_0)
    if m_3:
        q_1 = x(q_1)
    if m_2:
        q_1 = z(q_1)
    return q_1, m_2, m_3


In [8]:
# pytket approach
pytket_circ2 = convert_graphix_pattern_to_pytket(pattern2)

print("="*70)
print("PYTKET CIRCUIT:")
print("="*70)
print(f"Gates: {pytket_circ2.n_gates}, Depth: {pytket_circ2.depth()}")
print("Commands:")
for i, cmd in enumerate(pytket_circ2.get_commands()):
    print(f"  {i}: {cmd}")
print("="*70)

render_circuit_jupyter(pytket_circ2)

PYTKET CIRCUIT:
Gates: 11, Depth: 7
Commands:
  0: H q[1];
  1: H q[2];
  2: CZ q[0], q[1];
  3: Rz(0.5) q[0];
  4: CZ q[1], q[2];
  5: H q[0];
  6: H q[1];
  7: Measure q[0] --> m[0];
  8: Measure q[1] --> m[1];
  9: IF ([m[1]] == 1) THEN X q[2];
  10: IF ([m[0]] == 1) THEN Z q[2];


## Test 3: Rotation Gate (Rz)

Testing parametric gates with measurements and potential conditional corrections.

In [9]:
circuit3 = GraphixCircuit(1)
circuit3.rz(0, np.pi/4)
pattern3 = circuit3.transpile().pattern

print("Graphix Pattern Commands:")
for i, cmd in enumerate(pattern3):
    print(f"  {i}: {cmd}")
print()

Graphix Pattern Commands:
  0: N(1)
  1: N(2)
  2: E((0, 1))
  3: E((1, 2))
  4: M(0, angle=-0.25)
  5: M(1)
  6: X(2, {1})
  7: Z(2, {0})



In [10]:
# Original approach
guppy_code3 = convert_graphix_pattern_to_guppy(pattern3)
print("="*70)
print("GUPPY CODE:")
print("="*70)
print(guppy_code3)
print("="*70)

GUPPY CODE:
from guppy import guppy
from guppy.prelude.quantum import qubit, measure, h, x, y, z, s, sdg, rx, ry, rz, cz

@guppy
def quantum_circuit(q_in_0: qubit) -> tuple[qubit, bool, bool]:
    q_0 = qubit()  # Allocate qubit in |0⟩
    q_0 = h(q_0)  # Prepare |+⟩ state
    q_1 = qubit()  # Allocate qubit in |0⟩
    q_1 = h(q_1)  # Prepare |+⟩ state
    q_in_0, q_0 = cz(q_in_0, q_0)
    q_0, q_1 = cz(q_0, q_1)
    q_in_0 = rz(q_in_0, 0.25)
    q_in_0 = h(q_in_0)
    m_2 = measure(q_in_0)
    q_0 = h(q_0)
    m_3 = measure(q_0)
    if m_3:
        q_1 = x(q_1)
    if m_2:
        q_1 = z(q_1)
    return q_1, m_2, m_3


In [11]:
# pytket approach
pytket_circ3 = convert_graphix_pattern_to_pytket(pattern3)

print("="*70)
print("PYTKET CIRCUIT:")
print("="*70)
print(f"Gates: {pytket_circ3.n_gates}, Depth: {pytket_circ3.depth()}")
print("Commands:")
for i, cmd in enumerate(pytket_circ3.get_commands()):
    print(f"  {i}: {cmd}")
print("="*70)

render_circuit_jupyter(pytket_circ3)

PYTKET CIRCUIT:
Gates: 11, Depth: 7
Commands:
  0: H q[1];
  1: H q[2];
  2: CZ q[0], q[1];
  3: Rz(0.25) q[0];
  4: CZ q[1], q[2];
  5: H q[0];
  6: H q[1];
  7: Measure q[0] --> m[0];
  8: Measure q[1] --> m[1];
  9: IF ([m[1]] == 1) THEN X q[2];
  10: IF ([m[0]] == 1) THEN Z q[2];


## Test 4: Two-Qubit Gate (CNOT)

**This is where the conditional logic differences really show up!**

MBQC implementation of CNOT requires measurements and conditional corrections based on measurement outcomes.

In [12]:
circuit4 = GraphixCircuit(2)
circuit4.cnot(0, 1)
pattern4 = circuit4.transpile().pattern

print("Graphix Pattern Commands:")
for i, cmd in enumerate(pattern4):
    print(f"  {i}: {cmd}")
    # Highlight conditional corrections
    if hasattr(cmd, 'domain') and cmd.domain:
        print(f"       ⮕ CONDITIONAL on measurements: {cmd.domain}")
print()

Graphix Pattern Commands:
  0: N(2)
  1: N(3)
  2: E((1, 2))
  3: E((0, 2))
  4: E((2, 3))
  5: M(1)
  6: M(2)
  7: X(3, {2})
       ⮕ CONDITIONAL on measurements: {2}
  8: Z(3, {1})
       ⮕ CONDITIONAL on measurements: {1}
  9: Z(0, {1})
       ⮕ CONDITIONAL on measurements: {1}



### Original Approach: IF Statements in Generated Code

In [13]:
guppy_code4 = convert_graphix_pattern_to_guppy(pattern4)
print("="*70)
print("GUPPY CODE WITH IF STATEMENTS:")
print("="*70)
print(guppy_code4)
print("="*70)

# Highlight the conditional logic
print("\n🔍 CONDITIONAL LOGIC ANALYSIS:")
print("Look for 'if' statements in the code above.")
if_count = guppy_code4.count('if ')
print(f"Number of 'if' statements: {if_count}")

if if_count > 0:
    print("\nThese represent classical control flow - the gate application")
    print("is decided at runtime based on measurement outcomes.")

GUPPY CODE WITH IF STATEMENTS:
from guppy import guppy
from guppy.prelude.quantum import qubit, measure, h, x, y, z, s, sdg, rx, ry, rz, cz

@guppy
def quantum_circuit(q_in_0: qubit, q_in_1: qubit) -> tuple[qubit, qubit, bool, bool]:
    q_0 = qubit()  # Allocate qubit in |0⟩
    q_0 = h(q_0)  # Prepare |+⟩ state
    q_1 = qubit()  # Allocate qubit in |0⟩
    q_1 = h(q_1)  # Prepare |+⟩ state
    q_in_1, q_0 = cz(q_in_1, q_0)
    q_in_0, q_0 = cz(q_in_0, q_0)
    q_0, q_1 = cz(q_0, q_1)
    q_in_1 = h(q_in_1)
    m_2 = measure(q_in_1)
    q_0 = h(q_0)
    m_3 = measure(q_0)
    if m_3:
        q_1 = x(q_1)
    if m_2:
        q_1 = z(q_1)
    if m_2:
        q_in_0 = z(q_in_0)
    return q_in_0, q_1, m_2, m_3

🔍 CONDITIONAL LOGIC ANALYSIS:
Look for 'if' statements in the code above.
Number of 'if' statements: 3

These represent classical control flow - the gate application
is decided at runtime based on measurement outcomes.


### pytket Approach: Condition Arguments (No IF Statements!)

In [14]:
pytket_circ4 = convert_graphix_pattern_to_pytket(pattern4)

print("="*70)
print("PYTKET CIRCUIT WITH CONDITION ARGUMENTS:")
print("="*70)
print(f"Qubits: {pytket_circ4.n_qubits}")
print(f"Bits: {pytket_circ4.n_bits}")
print(f"Gates: {pytket_circ4.n_gates}")
print(f"Depth: {pytket_circ4.depth()}")

print("\nCommands (showing conditional gates):")
conditional_count = 0
for i, cmd in enumerate(pytket_circ4.get_commands()):
    # Check if command has a condition
    op = cmd.op
    print(f"  {i}: {cmd}")
    
    # pytket Commands don't directly expose conditions in the str representation,
    # but we can check the circuit's conditional gates
    
print("="*70)

# Analyze conditional gates
print("\n🔍 CONDITIONAL LOGIC ANALYSIS:")
print("pytket uses 'condition' kwargs instead of 'if' statements.")
print("Conditional gates are represented as circuit metadata.")
print("\nThis approach:")
print("  ✓ Integrates with TKET compilation passes")
print("  ✓ Can be converted to OpenQASM conditional syntax")
print("  ✓ Supports complex boolean expressions (XOR, AND, OR)")
print("  ✓ Backend-agnostic representation")

render_circuit_jupyter(pytket_circ4)

PYTKET CIRCUIT WITH CONDITION ARGUMENTS:
Qubits: 4
Bits: 2
Gates: 12
Depth: 8

Commands (showing conditional gates):
  0: H q[2];
  1: H q[3];
  2: CZ q[1], q[2];
  3: CZ q[0], q[2];
  4: H q[1];
  5: Measure q[1] --> m[0];
  6: CZ q[2], q[3];
  7: IF ([m[0]] == 1) THEN Z q[0];
  8: H q[2];
  9: Measure q[2] --> m[1];
  10: IF ([m[1]] == 1) THEN X q[3];
  11: IF ([m[0]] == 1) THEN Z q[3];

🔍 CONDITIONAL LOGIC ANALYSIS:
pytket uses 'condition' kwargs instead of 'if' statements.
Conditional gates are represented as circuit metadata.

This approach:
  ✓ Integrates with TKET compilation passes
  ✓ Can be converted to OpenQASM conditional syntax
  ✓ Supports complex boolean expressions (XOR, AND, OR)
  ✓ Backend-agnostic representation


## Test 5: More Complex Circuit

Let's create a circuit with multiple gates to see more conditional corrections.

In [15]:
circuit5 = GraphixCircuit(2)
circuit5.h(0)
circuit5.cnot(0, 1)
circuit5.rz(1, np.pi/3)
pattern5 = circuit5.transpile().pattern

print("Graphix Pattern Commands:")
for i, cmd in enumerate(pattern5):
    print(f"  {i}: {cmd}")
    if hasattr(cmd, 'domain') and cmd.domain:
        print(f"       ⮕ CONDITIONAL on: {cmd.domain}")
print()

Graphix Pattern Commands:
  0: N(2)
  1: E((0, 2))
  2: M(0)
  3: X(2, {0})
       ⮕ CONDITIONAL on: {0}
  4: N(3)
  5: N(4)
  6: E((1, 3))
  7: E((2, 3))
  8: E((3, 4))
  9: M(1)
  10: M(3)
  11: X(4, {3})
       ⮕ CONDITIONAL on: {3}
  12: Z(4, {1})
       ⮕ CONDITIONAL on: {1}
  13: Z(2, {1})
       ⮕ CONDITIONAL on: {1}
  14: N(5)
  15: N(6)
  16: E((4, 5))
  17: E((5, 6))
  18: M(4, angle=-0.3333333333333333)
  19: M(5)
  20: X(6, {5})
       ⮕ CONDITIONAL on: {5}
  21: Z(6, {4})
       ⮕ CONDITIONAL on: {4}



In [16]:
# Original approach
guppy_code5 = convert_graphix_pattern_to_guppy(pattern5)
print("GUPPY CODE:")
print("="*70)
print(guppy_code5)
print("="*70)
print(f"\nNumber of 'if' statements: {guppy_code5.count('if ')}")

GUPPY CODE:
from guppy import guppy
from guppy.prelude.quantum import qubit, measure, h, x, y, z, s, sdg, rx, ry, rz, cz

@guppy
def quantum_circuit(q_in_0: qubit, q_in_1: qubit) -> tuple[qubit, qubit, bool, bool, bool, bool, bool]:
    q_0 = qubit()  # Allocate qubit in |0⟩
    q_0 = h(q_0)  # Prepare |+⟩ state
    q_in_0, q_0 = cz(q_in_0, q_0)
    q_in_0 = h(q_in_0)
    m_1 = measure(q_in_0)
    if m_1:
        q_0 = x(q_0)
    q_2 = qubit()  # Allocate qubit in |0⟩
    q_2 = h(q_2)  # Prepare |+⟩ state
    q_3 = qubit()  # Allocate qubit in |0⟩
    q_3 = h(q_3)  # Prepare |+⟩ state
    q_in_1, q_2 = cz(q_in_1, q_2)
    q_0, q_2 = cz(q_0, q_2)
    q_2, q_3 = cz(q_2, q_3)
    q_in_1 = h(q_in_1)
    m_4 = measure(q_in_1)
    q_2 = h(q_2)
    m_5 = measure(q_2)
    if m_5:
        q_3 = x(q_3)
    if m_4:
        q_3 = z(q_3)
    if m_4:
        q_0 = z(q_0)
    q_6 = qubit()  # Allocate qubit in |0⟩
    q_6 = h(q_6)  # Prepare |+⟩ state
    q_7 = qubit()  # Allocate qubit in |0⟩
    q_

In [17]:
# pytket approach
pytket_circ5 = convert_graphix_pattern_to_pytket(pattern5)

print("PYTKET CIRCUIT:")
print("="*70)
print(f"Qubits: {pytket_circ5.n_qubits}, Bits: {pytket_circ5.n_bits}")
print(f"Gates: {pytket_circ5.n_gates}, Depth: {pytket_circ5.depth()}")
print("="*70)

render_circuit_jupyter(pytket_circ5)

PYTKET CIRCUIT:
Qubits: 7, Bits: 5
Gates: 28, Depth: 17


## Comparison Summary

### Architectural Differences

| Aspect | Original (Guppy) | pytket |
|--------|------------------|--------|
| **Output Format** | String (Python code) | Circuit object |
| **Conditional Logic** | `if` statements | `condition` kwarg |
| **Classical Expressions** | Runtime Python `^` operator | `BitLogicExp` objects |
| **Representation** | Imperative | Declarative |
| **Backend Integration** | Manual translation needed | Native TKET support |

### When to Use Each

**Use Original (Guppy) approach when:**
- Target is direct code generation
- You need explicit control flow in output
- Generating code for specific runtime environments
- Maximum transparency in generated output

**Use pytket approach when:**
- Leveraging TKET ecosystem and compilation passes
- Need backend-agnostic circuit representation
- Want to optimize circuits before execution
- Targeting multiple quantum hardware backends
- Need to convert to OpenQASM or other formats

### Key Insight

Both approaches correctly implement MBQC conditional corrections, but:
- **Original**: Generates explicit control flow (procedural)
- **pytket**: Embeds conditions as metadata (declarative)

The pytket approach is more aligned with how modern quantum circuit frameworks represent conditional operations.

## Export pytket Circuit to OpenQASM

Let's see how pytket's conditional gates translate to OpenQASM format.

In [18]:
from pytket.qasm import circuit_to_qasm_str

try:
    # Try to convert the CNOT circuit to OpenQASM
    qasm_str = circuit_to_qasm_str(pytket_circ4, header="hqslib1")
    print("="*70)
    print("OPENQASM OUTPUT:")
    print("="*70)
    print(qasm_str)
    print("="*70)
    print("\nNote: Conditional gates appear as 'if' statements in OpenQASM,")
    print("but they're represented declaratively in the pytket Circuit.")
except Exception as e:
    print(f"Could not convert to OpenQASM: {e}")
    print("This is expected for circuits with complex conditional logic.")

OPENQASM OUTPUT:
OPENQASM 2.0;
include "hqslib1.inc";

qreg q[4];
creg m[2];
h q[2];
h q[3];
cz q[1],q[2];
cz q[0],q[2];
h q[1];
measure q[1] -> m[0];
cz q[2],q[3];
if(m[0]==1) z q[0];
h q[2];
measure q[2] -> m[1];
if(m[1]==1) x q[3];
if(m[0]==1) z q[3];


Note: Conditional gates appear as 'if' statements in OpenQASM,
but they're represented declaratively in the pytket Circuit.


## Conclusion

We've compared two valid approaches to translating MBQC patterns:

1. **Code Generation** (original): Produces explicit imperative code with control flow
2. **Circuit Representation** (pytket): Uses declarative circuit objects with condition metadata

Both correctly implement the classical conditional logic required for MBQC, but they serve different purposes:
- The original is better for **code generation** and **transparency**
- The pytket approach is better for **compilation** and **backend interoperability**

The choice depends on your use case!